### Импорт библиотек

In [5]:
import matplotlib.pyplot as plt

from download_data import *
from prepare_features import *
from train import *
from submit_utils import *

### Загрузка данных

В качестве обучающих данных использовались данные из репозитория https://github.com/CSSEGISandData/COVID-19/archive/master.zip. А также дополнительные данные из открытых источников: данные о странах, количество населения по возрастным группам, количество курящего населения.

In [2]:
download_new_data(INPUT_PATH, INPUT_PATH)
check_latest_date()

Latest date available in time_series_covid19_confirmed_global.csv: 12/16/20


### Первоначальная обработка данных

Первоначальная обработка заключается в генерации базовых данных для обучения. Для каждого из дней готовилась отдельная модель, для которой был зафиксирован отдельных интервал данных за предыдущие 10 дней, на основании которых генерировались признаки для обучения

target | name |    date    | m0 | m1  | m2  | m3  | m4  | m5 | m6  | m7 | m8  | m9

714 | AFG | 2020.10.06 | 607 | 555 | 521 | 484 | 444 | 423 | 367 | 349 | 299 | 281

665 | AFG | 2020.10.05 | 555 | 521 | 484 | 444 | 423 | 367 | 349 | 299 | 281 | 273

607 | AFG | 2020.10.04 | 521 | 484 | 444 | 423 | 367 | 349 | 299 | 281 | 273 | 237

555 | AFG | 2020.04.09 | 484 | 444 | 423 | 367 | 349 | 299 | 281 | 273 | 237 | 174

521 | AFG | 2020.04.08 | 444 | 423 | 367 | 349 | 299 | 281 | 273 | 237 | 174 | 170

484 | AFG | 2020.04.07 | 423 | 367 | 349 | 299 | 281 | 273 | 237 | 174 | 170 | 120

444 | AFG | 2020.04.06 | 367 | 349 | 299 | 281 | 273 | 237 | 174 | 170 | 120 | 110

In [3]:
convert_timeseries_last_date(INPUT_PATH,FEATURES_PATH)

In [4]:
convert_timeseries_confirmed(INPUT_PATH, FEATURES_PATH, 'confirmed', False)
convert_timeseries_confirmed(INPUT_PATH, FEATURES_PATH, 'deaths', False)

In [5]:
for i in range(0, DAYS_TO_PREDICT + 1):
    create_time_features_confirmed(i, 'confirmed', FEATURES_PATH)
    create_time_features_confirmed(i, 'deaths', FEATURES_PATH)

In [6]:
get_first_case_date('confirmed', INPUT_PATH, FEATURES_PATH)
get_first_case_date('deaths', INPUT_PATH, FEATURES_PATH)

### Обучение и предсказание

В качестве модели использовался CatBoostRegressor со следующими параметрами:
* loss_function: RMSE,
* eval_metric: RMSE,
* iterations: 10000,
* learning_rate: random.choice([0.01, 0.03, 0.05]),
* depth: random.choice([4, 5, 6]),
* bootstrap_type: Bayesian,
* l2_leaf_reg: 3


Обучение проводилось на num_folds = random.randint(4, 5)

В качестве признаков использовались следующие:
1) log10([X] + 1)


2) (m[X] + 1) / (m[X-1] + 1)


3) (log10(m[X] + 1) + 1) / (log10(m[X-1] + 1) + 1)


4) Интерполяция


5) Стандартные признаки для стран:


Плотность населения, площадь, медианный возраст, мигранты, численность населения, городское население, среднее число детей в семье.


Численность населения по каждой стране с разбиением по возрастам “PopMale-0-19, PopFemale-0-19, PopMale-20-39, PopFemale-20-39, PopMale-40-59, PopFemale-40-59, PopMale-60+, PopFemale-60+”.


Количество курящего населения по странам


6) Дополнительные признаки
День недели
Количество дней с первого зарегистрированного случая заражения
Индекс самоизоляции от Яндекса по дням (10 последних доступных)



In [7]:
start_time = time.time()
gbm_type = 'CatB'
params = get_params()
target = params['target']
id = params['id']
metric = params['metric']
limit_date = DAYS_TO_PREDICT

all_scores = dict()
alldays_preds_train = dict()
alldays_preds_test = dict()
for type in ['confirmed', 'deaths']:
    print('Go for type: {}'.format(type))
    alldays_preds_train[type] = []
    alldays_preds_test[type] = []
    for day in range(1, limit_date+1):
        train, test, features = read_input_data(day, type, step_back_days=STEP_BACK)
        print('Features: [{}] {}'.format(len(features), features))
        print('Test date: {}'.format(sorted(test['date'].unique())))

        if 0:
            train[features].to_csv(CACHE_PATH + 'train_debug_{}_{}.csv'.format(type, day))
            test[features].to_csv(CACHE_PATH + 'test_debug_{}_{}.csv'.format(type, day))

        overall_train_predictions, score, model_list, importance = create_catboost_model(train, features, params, day)
        prefix = '{}_day_{}_{}_{:.6f}'.format(gbm_type, day, len(model_list), score)
        save_in_file((score, model_list, importance, overall_train_predictions), MODELS_PATH + prefix + '.pklz')

        all_scores[(type, day)] = score
        train['pred'] = overall_train_predictions
        train['pred'] = np.maximum(train['pred'], train['case_day_minus_0'])
        train[['name1', 'name2', 'date', 'target', 'pred']].to_csv(SUBM_PATH + prefix + '_train.csv', index=False, float_format='%.8f')
        train_tmp = decrease_table_for_last_date(train)
        alldays_preds_train[type].append(train_tmp[['name1', 'name2', 'date', 'target', 'pred']].copy())

        overall_test_predictions = predict_with_catboost_model(test, features, model_list)
        test['pred'] = overall_test_predictions

        count_less = (test['pred'] < test['case_day_minus_0']).astype(np.int32).sum()
        if count_less > 0:
            print('Values less than needed for test: {} ({:.4f} %)'.format(count_less, 100 * count_less / len(test)))
        test['pred'] = np.maximum(test['pred'], test['case_day_minus_0'])

        test['shift_day'] = day
        test[['name1', 'name2', 'date', 'pred']].to_csv(SUBM_PATH + prefix + '_test.csv', index=False, float_format='%.8f')
        alldays_preds_test[type].append(test[['name1', 'name2', 'date', 'shift_day', 'pred']].copy())

    train = pd.concat(alldays_preds_train[type], axis=0)
    score = contest_metric(train['target'].values, train['pred'].values)
    all_scores[(type, 'full')] = score
    print('Total score {} for all days: {:.6f}'.format(type, score))
    prefix = '{}_{}_all_days_{}_{:.6f}'.format(gbm_type, type, len(model_list), score)
    train.to_csv(SUBM_PATH + '{}_train.csv'.format(prefix), index=False)
    test = pd.concat(alldays_preds_test[type], axis=0)
    test.to_csv(SUBM_PATH + '{}_test.csv'.format(prefix), index=False)

    prefix_2 = '{}_{}_LOG_{}_DIFF_{}_DIV_{}_countries'.format(gbm_type, type, USE_LOG, USE_DIFF, USE_DIV)
    train.to_csv(SUBM_PATH + '{}_train.csv'.format(prefix_2), index=False)
    test.to_csv(SUBM_PATH + '{}_test.csv'.format(prefix_2), index=False)

for type in ['confirmed', 'deaths']:
    for day in range(1, limit_date + 1):
        print('Type: {} Day: {} Score: {:.6f}'.format(type, day, all_scores[(type, day)]))
for type in ['confirmed', 'deaths']:
    print('Total score {} for all days: {:.6f}'.format(type, all_scores[(type, 'full')]))

print("Elapsed time overall: {:.2f} seconds".format((time.time() - start_time)))

Go for type: confirmed
60929 61120
Removed zero target. Reduction 60929 -> 53766
53766 169
53766 169
Features: [96] ['case_day_minus_0', 'case_day_minus_1', 'case_day_minus_2', 'case_day_minus_3', 'case_day_minus_4', 'case_day_minus_5', 'case_day_minus_6', 'case_day_minus_7', 'case_day_minus_8', 'case_day_minus_9', 'case_deaths_day_minus_0', 'case_deaths_day_minus_1', 'country', 'case_day_minus_0_log10', 'case_day_minus_1_log10', 'case_day_minus_2_log10', 'case_day_minus_3_log10', 'case_day_minus_4_log10', 'case_day_minus_5_log10', 'case_day_minus_6_log10', 'case_day_minus_7_log10', 'case_day_minus_8_log10', 'case_day_minus_9_log10', 'diff_div_0', 'diff_div_0_opp', 'diff_div_1', 'diff_div_1_opp', 'diff_div_2', 'diff_div_2_opp', 'diff_div_3', 'diff_div_3_opp', 'diff_div_4', 'diff_div_4_opp', 'diff_div_5', 'diff_div_5_opp', 'diff_div_6', 'diff_div_6_opp', 'diff_div_7', 'diff_div_7_opp', 'diff_div_8', 'diff_div_8_opp', 'diff_log_div_0', 'diff_log_div_0_opp', 'diff_log_div_1', 'diff_log_di

### Графики показывающие финальные предсказания а также описание важности признаков

case_day_minus_0: 28.665248

case_day_minus_0_log10: 23.761416

interpol_1: 19.050890

country: 7.001849

case_day_minus_1_log10: 4.611567

case_day_minus_1: 2.380624

case_day_minus_2: 1.578850

interpol_log_1: 1.461800

interpol_2: 1.132108

case_confirmed_day_minus_0: 1.121451

case_day_minus_9_log10: 1.104196

case_confirmed_day_minus_1: 1.056181

### Итоговые графики для России

In [14]:
Подтвержденные случаи

In [15]:
Смерти

### Итоговые графики для США

In [18]:
Смерти

In [17]:
Подтвержденные случаи

### Выводы

Модель бустинга показала себя более устойчивой к нестабильности данных, чем модель SEIR. Однако она обладает ограниченным обобщающим потенциалом, что может служить препятствием для использования её в будущих предсказаниях